# Stopword들을 찾고 제거하거나 1개로 통일하자

## 양 옆에 공백을 두고 있지만 의미없는 단어

In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
from collections import defaultdict

In [2]:
def make_dataframe(path: str) -> pd.DataFrame:
    """
    Read a json file and return a pandas DataFrame.

    Parameters:
    path (str): Path to the json file.

    Returns:
    pd.DataFrame: DataFrame of the json file.
    """
    # Read the json file
    with open(path, 'r') as file:
        data = json.load(file)

    # Create a DataFrame
    # columns = ['id', 'conversation', 'subject_keyword', 'output']
    df = pd.DataFrame(data)
    df['conversation'] = df['input'].apply(lambda x: x['conversation'])
    df['subject_keyword'] = df['input'].apply(lambda x: x['subject_keyword'])

    # Drop the 'input' column
    df.drop('input', axis=1, inplace=True)

    # Speakers in the conversation
    df['speakers'] = df['conversation'].apply(lambda turns: list(set(turn['speaker'] for turn in turns)))

    # Reorder the columns
    df = df[['id', 'conversation', 'subject_keyword', 'speakers', 'output']]

    return df

In [3]:
# train_df = make_dataframe('../resource/data/일상대화요약_train.json')
# dev_df = make_dataframe('../resource/data/일상대화요약_dev.json')
# test_df = make_dataframe('../resource/data/일상대화요약_test.json')
train_df = make_dataframe('./train.json')
dev_df = make_dataframe('./dev.json')
test_df = make_dataframe('./test.json')

In [11]:
# find the stopwords
def find_stopwords(df: pd.DataFrame, pattern) -> set:
    """
    Find the stopwords in the DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame of the json file.

    Returns:
    set: Set of stopwords.
    """
    # Find the stopwords
    stopwords = defaultdict(int)
    stopwords_in_output = defaultdict(int)
    utter_cnt = 0
    output_cnt = 0

    for idx, row in df.iterrows():
        # Find the stopwords in the output
        output = row['output']
        stopwords_list = re.findall(pattern, output)
        for stopword in stopwords_list:
            stopwords_in_output[stopword] += 1
            output_cnt+=1


        # Find the stopwords in the conversation
        for turn in row['conversation']:
            utterance = turn['utterance']

            # Find the stopwords and add them to stopwords
            stopwords_list = re.findall(pattern, utterance)
            for stopword in stopwords_list:
                stopwords[stopword] += 1
                utter_cnt+=1

    # Make a Series of stopwords
    stopwords = pd.Series(stopwords)
    stopwords_in_output = pd.Series(stopwords_in_output)

    print(f"Number of stopwords in utterance : {len(stopwords)}")
    print(f"Number of stopwords in output : {len(stopwords_in_output)}")
    print(f"Frequency of stopwords in utterance : {utter_cnt}")
    print(f"Frequency of stopwords in output : {output_cnt}")
    

    return stopwords, stopwords_in_output

In [12]:
stopwords, st_output = find_stopwords(train_df, pattern = r'(?:\s+[가-힣]{1}\s+)')

Number of stopwords in utterance : 694
Number of stopwords in output : 223
Frequency of stopwords in utterance : 40384
Frequency of stopwords in output : 3801


In [132]:
stopwords, st_output = find_stopwords(test_df, pattern = r'\b좀\b')

Number of stopwords: 1
Number of stopwords in the output: 0


In [133]:
stopwords

좀    872
dtype: int64

In [130]:
stopwords.loc['좀']

2796

In [131]:
st_output.loc['좀']

19

In [65]:
stopwords.sort_values(ascending=False, inplace=True)

In [66]:
stopwords[:50]

 거     5282
 좀     2534
 안     2465
 또     2339
 게     2235
 한     1767
 때     1438
 잘     1286
 다     1281
 더     1055
 수      869
 못      782
 할      768
 해      712
 걸      669
 것      627
 이      626
 건      529
 내      510
 나      414
 난      336
 가      326
 될      318
 꼭      315
 왜      308
 참      304
 갈      291
 딱      285
 제      276
 데      270
 몇      255
 본      243
 저      242
 두      225
 큰      223
 볼      217
 사      172
 너      158
 줄      154
 된      153
 집      143
 살      141
 뭘      137
 날      134
 원      130
 네      129
 적      128
 세      117
 돼      113
 일      105
dtype: int64

In [24]:
import re

# 테스트 문자열들
texts = [
    "그래 그래서",   # 매칭됨
    "아니 아니야",   # 매칭됨
    "하지 하지만",   # 매칭됨
    "먹먹 먹",       # 매칭되지 않음
    "그래 그래",     # 매칭되지 않음
    "응 응해",       # 매칭됨
]

# 정규 표현식 패턴
pattern = r'\b(\w+)\s+\1(\w*\b)'

# 각 문자열에 대해 패턴과 일치하는지 확인
for text in texts:
    match = re.search(pattern, text)
    if match:
        print(f"Match found in '{text}':", match.group())
    else:
        print(f"No match found in '{text}'")

Match found in '그래 그래서': 그래 그래서
Match found in '아니 아니야': 아니 아니야
Match found in '하지 하지만': 하지 하지만
No match found in '먹먹 먹'
Match found in '그래 그래': 그래 그래
Match found in '응 응해': 응 응해


In [115]:
sw, sw_output = find_stopwords(train_df, pattern = r'\b\s+(\w+)\s+\1(\w*)\b')

Number of stopwords: 901
Number of stopwords in the output: 5


In [116]:
sw_output

두루        1
할머니가      1
깜짝        1
좋은        1
가도        1
dtype: int64

In [117]:
sw.sort_values(ascending=False, inplace=True)
sw[:50]

가    지고     41
있    는       5
내    가       5
하    고       4
많    이       4
시국이  니만큼     3
거    든       3
제    대로      3
하    는       3
이    런       3
그렇   게       3
이    게       3
나    도       3
     중에      3
물    론       2
옷    을       2
다    니고      2
있    을       2
나    는       2
것    도       2
거기   가       2
그래   서       2
다    녔던      2
요    런       2
가    장       2
없    는       2
거    기       2
다    르고      2
사    실       2
학    교       2
다    르게      2
그거   를       2
키우   는       2
되    면       2
우    리       2
마    라톤      2
너    도       2
고    양이가     2
먹    을       2
있    어       2
대    충       2
일    단       2
다    른데      2
있    어서      2
이    것도      2
영    화를      2
있    다       2
언    제야      2
요즘   은       2
지금   도       2
dtype: int64

In [118]:
samples = ["그럼 네가 넷플릭스에서 봤었던 넷플릭스 영화만 있 있는 것도 있잖아.", # 있 있는 : train 5,
           " 근데 우리 강아지는 내 내가 중학교 2학년 땐가 집을 잠깐 비운 적 있었는데", # 내 내가 : train 5
           "안 먹이려고 하고 있 하 하고 있고요.", # 하 하고: train 4
           "엄청 많 많이 본 거 같아.", # 많 많이: train 4
           "영화여서 되게 좋아 하 하는 거 같아.", # 하 하는: train 3
           "육전도 들어가고 그렇 그렇게 하면서", # 그렇 그렇게: train 3
           "시즌 투 나왔을 때도 나 나중에 본다 나중에 아껴볼 거라고", # 나 나중에: train 3
           "그래 그래서 애니메이션이 진격의 거인이었 거인이었거든요.", # 거인이었 거인이: train 3
           ]

for sample in samples:
    print(re.findall(r'\b(\w+)\s+\1(\w*)\b', sample))

[('있', '는')]
[('내', '가')]
[('하', '고')]
[('많', '이')]
[('하', '는')]
[('그렇', '게')]
[('나', '중에')]
[('그래', '서'), ('거인이었', '거든요')]


In [39]:
re.sub(r'\b([가-힣]+)\s+\1([가-힣]*)\b', r'\1\2','그래 그래서 애니메이션이 진격의 거인이었 거인이었거든요.')

'그래서 애니메이션이 진격의 거인이었거든요.'

In [71]:
import re

# 테스트 문자열들
texts = [
    "그래 그래서",   # 매칭됨
    "아니 아니야",   # 매칭됨
    "하지 하지만",   # 매칭됨
    "먹먹 먹",       # 매칭되지 않음
    "그래 그래",     # 매칭되지 않음
    "응 응해",       # 매칭됨
]

# 정규 표현식 패턴
pattern = r'\b([가-힣]+)\s+\1([가-힣]+)\b'

# 각 문자열에 대해 패턴과 일치하는지 확인하고 치환
for text in texts:
    result = re.sub(pattern, r'\1\2', text)
    print(f"Original: '{text}' => Modified: '{result}'")

Original: '그래 그래서' => Modified: '그래서'
Original: '아니 아니야' => Modified: '아니야'
Original: '하지 하지만' => Modified: '하지만'
Original: '먹먹 먹' => Modified: '먹먹 먹'
Original: '그래 그래' => Modified: '그래 그래'
Original: '응 응해' => Modified: '응해'


## X를 포함하는 단어들

In [10]:
find_stopwords(test_df, pattern = r'\b[가-힣a-zA-Z]*[xX][가-힣a-zA-Z]*\b')

Number of stopwords: 59


xxx          4
xx로          1
x           28
x는           2
제x           1
다니x          1
즐겁더라고x       1
xx케          1
x케           1
xx          15
요즘x          1
어디xx         1
요x           1
이x           3
마마마마x        1
근x           1
싶x           1
생각xx         1
알았x          1
같x           2
운x을          1
그x           2
같으xx요        1
그래x          1
왜x면          1
그xx          1
x이지만         1
그래갖고서x       1
먹x           1
그x고          1
끊x고          1
x다음에         1
xx서          6
한x           1
x번           1
xx으로         1
올라x고         1
x보적으로        1
아이x          1
xxxxx습니다     1
x거           1
xx도          1
x짜           1
무난했xx        1
xxxx         2
x데           4
그x까          1
같xx          1
xx게          1
x시           1
해x고          1
꼽으라x         1
x니까          4
x게           1
xx까          1
싶xx          1
지난xx         1
xx튼          1
x까           1
dtype: int64

In [13]:
find_stopwords(test_df, pattern = r'\b\w*[xX]\w*\b')

Number of stopwords: 59


xxx          4
xx로          1
x           28
x는           2
제x           1
다니x          1
즐겁더라고x       1
xx케          1
x케           1
xx          15
요즘x          1
어디xx         1
요x           1
이x           3
마마마마x        1
근x           1
싶x           1
생각xx         1
알았x          1
같x           2
운x을          1
그x           2
같으xx요        1
그래x          1
왜x면          1
그xx          1
x이지만         1
그래갖고서x       1
먹x           1
그x고          1
끊x고          1
x다음에         1
xx서          6
한x           1
x번           1
xx으로         1
올라x고         1
x보적으로        1
아이x          1
xxxxx습니다     1
x거           1
xx도          1
x짜           1
무난했xx        1
xxxx         2
x데           4
그x까          1
같xx          1
xx게          1
x시           1
해x고          1
꼽으라x         1
x니까          4
x게           1
xx까          1
싶xx          1
지난xx         1
xx튼          1
x까           1
dtype: int64

## 제어문자

In [13]:
find_stopwords(train_df, pattern=r'[\x00-\x1F\x7F]')

Number of stopwords in utterance : 0
Number of stopwords in output : 3
Frequency of stopwords in utterance : 0
Frequency of stopwords in output : 13


(Series([], dtype: object),
      5
 
      6
 \n    2
 dtype: int64)

- 전처리 이후

In [14]:
train_df = make_dataframe('./train.json')
dev_df = make_dataframe('./dev.json')

In [15]:
find_stopwords(train_df, pattern=r'[\x00-\x1F\x7F]')

Number of stopwords in utterance : 0
Number of stopwords in output : 0
Frequency of stopwords in utterance : 0
Frequency of stopwords in output : 0


(Series([], dtype: object), Series([], dtype: object))